In [0]:
!pip install transformers

     |████████████████████████████████| 675kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 13.1MB/s 
     |████████████████████████████████| 3.8MB 12.8MB/s 
     |████████████████████████████████| 890kB 41.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=afd26c1c42f92444ead9bc8f6d16f434e79836c80102d0bc9f367d013b89b42d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [0]:
import pandas as pd
import numpy as np
import transformers

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/kaggle-tweet-sentiment-extraction/alberta")
!pwd

/content/drive/My Drive/kaggle-tweet-sentiment-extraction/alberta


In [0]:
train = pd.read_csv("../input/train.csv")

In [0]:
train.shape

(27481, 4)

In [0]:
otrain = pd.read_csv("../input/tweet_dataset.csv")

In [0]:
otrain.shape

(40000, 8)

In [0]:
tokenizer = transformers.RobertaTokenizerFast.from_pretrained("roberta-base", do_lower_case=True)

In [0]:
import torch


def find_start_and_end(tweet, selected_text):
    len_st = len(selected_text)
    start = None
    end = None
    for ind in (i for i, e in enumerate(tweet) if e == selected_text[0]):
        if tweet[ind: ind+len_st] == selected_text:
            start = ind
            end = ind + len_st - 1
            break
    return start, end

def process_with_offsets(args, tweet, selected_text, sentiment, tokenizer):

    start_index, end_index = find_start_and_end(tweet, selected_text)

    char_targets = [0]*len(tweet)
    if start_index != None and end_index != None:
        for ct in range(start_index, end_index+1):
            char_targets[ct] = 1
    
    encoded = tokenizer.encode_plus(
                    sentiment,
                    tweet,
                    #max_length=args.max_seq_len,
                    #pad_to_max_length=True,
                    return_token_type_ids=True,
                    return_offsets_mapping=True
                )

    target_idx = []
    for j, (offset1, offset2) in enumerate(encoded["offset_mapping"]):
        if j > 3:
            if sum(char_targets[offset1:offset2]) > 0:
                target_idx.append(j)

    encoded["tokens"] = tokenizer.tokenize(tweet) 

    encoded["start_position"] = target_idx[0]
    encoded["end_position"] = target_idx[-1]
    encoded["tweet"] = tweet
    encoded["selected_text"] = selected_text
    encoded["sentiment"] = sentiment

    return encoded

In [0]:
class args:
    max_seq_len = 100

In [0]:
for i, (tweet,selected_text, sentiment) in enumerate(zip(train.text.values, train.selected_text.values, train.sentiment.values)):
    
    features = process_with_offsets(args, tweet, selected_text, sentiment, tokenizer)

    print(features["tweet"])
    print(features["selected_text"])
    print(features["sentiment"])
    print(features["tokens"])
    print(features["input_ids"])

    print(features["start_position"], features["end_position"])

    print("="*100)
    
    if i == 100:
        break

 I`d have responded, if I were going
I`d have responded, if I were going
neutral
['ĠI', '`', 'd', 'Ġhave', 'Ġresponded', ',', 'Ġif', 'ĠI', 'Ġwere', 'Ġgoing']
[0, 7974, 2, 2, 38, 12905, 417, 33, 2334, 6, 114, 38, 58, 164, 2]
4 13
 Sooo SAD I will miss you here in San Diego!!!
Sooo SAD
negative
['ĠSo', 'oo', 'ĠS', 'AD', 'ĠI', 'Ġwill', 'Ġmiss', 'Ġyou', 'Ġhere', 'Ġin', 'ĠSan', 'ĠDiego', '!!!']
[0, 2430, 2, 2, 407, 3036, 208, 2606, 38, 40, 2649, 47, 259, 11, 764, 3402, 16506, 2]
4 7
my boss is bullying me...
bullying me
negative
['Ġmy', 'Ġboss', 'Ġis', 'Ġbullying', 'Ġme', '...']
[0, 2430, 2, 2, 127, 3504, 16, 11902, 162, 734, 2]
7 8
 what interview! leave me alone
leave me alone
negative
['Ġwhat', 'Ġinterview', '!', 'Ġleave', 'Ġme', 'Ġalone']
[0, 2430, 2, 2, 99, 1194, 328, 989, 162, 1937, 2]
7 9
 Sons of ****, why couldn`t they put them on the releases we already bought
Sons of ****,
negative
['ĠSons', 'Ġof', 'Ġ****', ',', 'Ġwhy', 'Ġcouldn', '`', 't', 'Ġthey', 'Ġput', 'Ġthem', 'Ġon', 'Ġthe'

In [0]:
tokenizer.encode("extract positive sentiment")

[0, 14660, 1313, 5702, 2]

In [0]:
tokenizer = transformers.AlbertTokenizer.from_pretrained("albert-xlarge-v2")

In [0]:
len(tokenizer)

30000

In [0]:
vocab_tokens = tokenizer.get_vocab().keys()

In [0]:
vocab_tokens

dict_keys(['<pad>', '<unk>', '[CLS]', '[SEP]', '[MASK]', '(', ')', '"', '-', '.', '–', '£', '€', '▁', '▁the', ',', '▁of', '▁and', 's', '▁in', '▁to', '▁a', "'", '▁was', '▁he', '▁is', '▁for', '▁on', '▁as', '▁with', '▁that', '▁i', '▁it', '▁his', '▁by', '▁at', '▁her', '▁from', 't', '▁she', '▁an', '▁had', '▁you', 'd', '▁be', ':', '▁were', '▁but', '▁this', 'i', '▁are', '▁my', '▁not', '▁one', '▁or', '▁me', '▁which', '▁have', 'a', '▁they', '?', '▁him', 'e', '▁has', '▁first', '▁all', '▁their', '▁also', 'ing', 'ed', '▁out', '▁up', '▁who', ';', '▁been', '▁after', '▁when', '▁into', '▁new', 'm', '▁there', '▁two', '▁its', '▁would', '▁over', '▁time', '▁so', '▁said', '▁about', '▁other', '▁no', '▁more', '▁can', 'y', '▁then', '▁we', 'th', '▁back', '▁what', 're', '▁if', '▁like', 'ly', 'n', '▁only', '▁them', 'er', '▁do', 'in', '▁some', '▁could', 'o', '▁during', '▁where', '▁just', '▁before', '▁school', '▁made', '/', '▁than', '▁through', '▁de', '▁years', '▁may', 'the', '▁down', '▁world', '▁most', '▁between'

In [0]:
all_tokens = []
all_ids = []
unk_tokens = []
for text in train.text.values:
    tokens = tokenizer.tokenize(str(text))
    ids = tokenizer.encode(str(text), add_special_tokens=False)
    if tokenizer.unk_token_id in ids:
        
        index = [i for i, e in enumerate(ids) if e == tokenizer.unk_token_id]
        unk_tokens.extend(list(np.array(tokens)[index]))

        print(text)
        print(tokens)
        print(tokenizer.convert_ids_to_tokens(ids))
        print(ids)
        print("="*100)
    all_tokens.extend(tokens)
    all_ids.extend(ids)

Streaming output truncated to the last 5000 lines.
['▁going', '▁out', '▁on', '▁thursday', 's', '▁is', '▁a', '▁terrible', '▁idea', '.', '▁i', '▁knew', '▁there', '▁was', '▁a', '▁reason', '▁i', '▁haven', '`', 't', '▁done', '▁it', '▁all', '▁quarter', '.', '▁*', 'pop', 's', '▁a', 'd', 'vil', '*']
['▁going', '▁out', '▁on', '▁thursday', 's', '▁is', '▁a', '▁terrible', '▁idea', '.', '▁i', '▁knew', '▁there', '▁was', '▁a', '▁reason', '▁i', '▁haven', '<unk>', 't', '▁done', '▁it', '▁all', '▁quarter', '.', '▁*', 'pop', 's', '▁a', 'd', 'vil', '*']
[228, 70, 27, 8775, 18, 25, 21, 5803, 882, 9, 31, 404, 80, 23, 21, 1215, 31, 2933, 1, 38, 677, 32, 65, 2650, 9, 1637, 6057, 18, 21, 43, 9638, 2483]
@_Emily_Young_ okey
['▁', '@_', 'emi', 'ly', '_', 'young', '_', '▁o', 'key']
['▁', '<unk>', 'emi', 'ly', '<unk>', 'young', '<unk>', '▁o', 'key']
[13, 1, 7411, 102, 1, 10886, 1, 635, 4237]
#bgt It`s a shame that the Dreambears won`t get through
['▁#', 'bg', 't', '▁it', '`', 's', '▁a', '▁shame', '▁that', '▁the', '

In [0]:
text = ""
for i in pd.Series(unk_tokens).value_counts().index:
    text += i

In [0]:
unk_tokens_last = []
for char in text:
    unk_tokens_last.append(char)

In [85]:
set(unk_tokens_last)

{'<', '>', '@', '\\', '^', '_', '`', '{', '|', '}', '~', '¿', '⁄'}

In [0]:
un_tokens = []
for tok in all_tokens:
    if tok == "<unk>":
        print(tok)

    

In [0]:
all_tokens_series = pd.Series(all_tokens)
all_ids_series = pd.Series(all_ids)

In [0]:
tokenizer.unk_token_id

1

In [0]:
tokenizer.encode("!@#$%^&*()123")

[0, 27785, 1039, 49255, 207, 35227, 947, 3226, 43048, 19644, 2]

In [0]:
all_ids_series[all_ids_series == 1].count()

14456

In [0]:
a

In [0]:
ind = [i for i, e in enumerate([1, 2, 1,1 ,1 ,1]) if e == 1]

In [0]:
np.array([1, 2, 1,1 ,1 ,1])[ind]

array([1, 1, 1, 1, 1])

In [0]:
[1,2,3,4,5,1].index(1)

0

In [0]:
all_ids_series.value_counts()

9        35815
31       16432
13       14903
1        14456
20       10188
         ...  
9450         1
13544        1
9038         1
23617        1
4453         1
Length: 15354, dtype: int64

In [0]:
all_tokens_series.value_counts()

.            15677
Ġ            13380
`            11292
ĠI           11284
Ġto           9941
             ...  
Ġthreat          1
iots             1
cash             1
ĠMister          1
ĠConflict        1
Length: 21549, dtype: int64

In [0]:
all_tokens_series[all_tokens_series == "<unk>"]

Series([], dtype: object)

In [0]:
tokenizer.unk_token

'<unk>'

In [0]:
import transformers
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class TweetModel(transformers.BertPreTrainedModel):
    def __init__(self, model_path, conf):
        super(TweetModel, self).__init__(conf)
        self.roberta = transformers.RobertaModel.from_pretrained(model_path, config=conf)
        self.drop_out = nn.Dropout(0.1)
        self.l0 = nn.Linear(768 * 7, 2) #768
        #self.l1 = nn.Linear(768 * 4, 1)
        #torch.nn.init.normal_(self.l0.weight, std=0.02)
        #self.init_weights()


    def forward(self,input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        out1,out2, out = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        #print(out1.shape)
        #print(out2.shape)
        #print(len(out))
        #print(out)

        hidden_list = list(out)
        hidden_stack = torch.stack(hidden_list, dim=0)

        SUM = torch.sum(hidden_stack, dim=0)
        MEAN = torch.mean(hidden_stack, dim=0)
        #MAX = torch.max(hidden_stack, dim=0)
        #MIN = torch.min(hidden_stack, dim=0)
        #MEAN = torch.stack(hidden_list, dim=
        #print(SUM.shape, MEAN.shape, MAX, MIN)


        out = torch.cat((out[-1], out[-2], out[-3], out[-4], out1, SUM, MEAN), dim=-1)
        #print(out.shape)
        out = self.drop_out(out)
        logits = self.l0(out)

        start_logits, end_logits = logits.split(1, dim=-1)

        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        #print(start_logits.shape, end_logits.shape)

        return start_logits, end_logits

In [0]:
model_path = "roberta-base"
config = transformers.RobertaConfig.from_pretrained(model_path)
config.output_hidden_states = True
tokenizer = transformers.RobertaTokenizerFast.from_pretrained("roberta-base", do_lower_case=True)

MX = TweetModel(model_path, config)

In [0]:
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)

In [0]:
MX(input_ids)

torch.Size([1, 8]) torch.Size([1, 8])


(tensor([[ 0.5260, -0.6225, -0.5750, -2.0465, -3.6607, -1.0610, -2.5060, -0.7185]],
        grad_fn=<SqueezeBackward1>),
 tensor([[ 2.3548, -0.0484, -0.4354, -0.4317, -1.3500, -1.1079,  0.1045,  1.3534]],
        grad_fn=<SqueezeBackward1>))

In [0]:
out = torch.rand((1, 8, 768))

In [0]:
my_list = (out,out)
#result = torch.stack(my_list, dim=0).sum(dim=0).sum(dim=0)

In [0]:
my_list = list(my_list)

In [0]:
results = torch.stack(my_list, dim=0)

In [0]:
results

torch.return_types.max(...)

In [0]:
torch.sum()

In [0]:
import sentencepiece as sp

In [0]:
torch.adaptive_max_pool1d(results, output_size=(1, 8, 768))

RuntimeError: ignored

In [0]:
torch.max